# Modèle Deep Learning (Embedding + LSTM)

### Sommaire : 
 * Importation des packages
 * Importation des données
 * Nettoyage des textes
 * Tokenisation et création de séquences
 * Préparation des labels (multi-label, 6 classes)
 * Séparation en jeux d'entraînement et de validation
 * Modèle A – MLP avec embeddings seuls
 * Création du modèle de MPL
 * Entraînement du modèle MLP
 * Évaluation sur le jeu de validation
 * Test du modèle MLP sur le dataset test.csv
 * Modèle B – RNN avec LSTM
 * Création du modèle de RNN avec LSTM
 * Entraînement du modèle LSTM
 * Évaluation sur le jeu de validation
 * Test du modèle RNN avec LSTM sur le dataset test.csv

# Importation des packages

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense, Dropout, LSTM
from tensorflow.keras.callbacks import EarlyStopping

# 1. Importation des données

Ajoutez un raccourci de ce dossier à votre google drive :

https://drive.google.com/drive/folders/1mx-CAzT10YKrmxHfYDP_1Oef7PVGUr7s?usp=sharing

In [ ]:
# Importez le module de lecteur de Google.colab pour interagir avec Google Drive
from google.colab import drive

# Montez Google Drive vers le répertoire '/Content/Drive'
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Importation des données d'entraînement à partir de Google Drive
data = pd.read_csv('/content/drive/MyDrive/data_classification_commentaires_toxiques/train.csv')

# Affichage des premières lignes du DataFrame pour vérifier l'importation
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


# 2. Nettoyage des textes

In [ ]:
# fonction nettoyage du texte
def clean_text(text):
    text = text.lower()  # Passage en minuscules
    text = re.sub(r'\W+', ' ', text)  # Suppression des caractères spéciaux
    return text

# Appliquer la fonction de nettoyage à la colonne 'comment_text' du DataFrame
data['clean_comment'] = data['comment_text'].astype(str).apply(clean_text)

# 3. Tokenisation et création de séquences

In [ ]:
max_words = 20000  # Nombre maximal de mots à prendre en compte
max_len = 100      # Longueur maximale des séquences

# Initialisation du tokenizer avec un nombre maximal de mots et un token pour les mots hors vocabulaire
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')

# Apprentissage du tokenizer sur les commentaires nettoyés
tokenizer.fit_on_texts(data['clean_comment'])

# Conversion des commentaires en séquences de tokens
sequences = tokenizer.texts_to_sequences(data['clean_comment'])

# Application du padding pour uniformiser la longueur des séquences
data_seq = pad_sequences(sequences, maxlen=max_len)

# 4. Préparation des labels (multi-label, 6 classes)

In [ ]:
# Sélectionner les colonnes de labels pour la classification multi-label
y = data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

# 5. Séparation en jeux d'entraînement et de validation

In [ ]:
# Séparer les données en ensembles d'entraînement et de test
# X : les features (matrice TF-IDF)
# y : les labels (multi-labels)
# test_size=0.2 : 20% des données seront utilisées pour le test, 80% pour l'entraînement
# random_state=42 : pour assurer la reproductibilité des résultats
X_train, X_val, y_train, y_val = train_test_split(data_seq, y, test_size=0.2, random_state=42)

# Callback pour arrêter l'entraînement si la perte de validation ne s'améliore plus
# monitor='val_loss' : surveille la perte sur le jeu de validation
# patience=2 : arrête l'entraînement après 2 epochs sans amélioration
# restore_best_weights=True : restaure les poids du modèle au meilleur epoch de validation
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# 6.1 **Modèle A – MLP avec embeddings seuls**
## 6.12 Création du modèle de MPL

In [ ]:
# Création du modèle MLP (Multi-Layer Perceptron)
model_mlp = Sequential([
    # Couche d'embedding pour convertir les indices de mots en vecteurs d'embedding
    Embedding(input_dim=max_words, output_dim=128),  
    
    # Couche de pooling global pour réduire la dimensionnalité en prenant la moyenne des embeddings
    GlobalAveragePooling1D(),                        
    
    # Couche dense avec 64 neurones et activation ReLU
    Dense(64, activation='relu'),
    
    # Couche de dropout pour éviter le surapprentissage en désactivant aléatoirement 50% des neurones
    Dropout(0.5),
    
    # Couche de sortie avec 6 neurones (une pour chaque classe) et activation sigmoïde pour la classification multi-label
    Dense(6, activation='sigmoid')                   
])

# Compilation du modèle avec l'optimiseur Adam et la fonction de perte binaire cross-entropie
model_mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Affichage du résumé du modèle créé
print("Résumé du modèle MLP :")
model_mlp.summary()

Résumé du modèle MLP :


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ ?                           │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## 6.13 Entraînement du modèle MLP

In [ ]:
# Entraînement du modèle MLP avec les données d'entraînement
history_mlp = model_mlp.fit(
    X_train,  # Les séquences de tokens pour l'entraînement
    y_train,  # Les labels multi-labels pour l'entraînement
    epochs=10,  # Nombre d'époques (cycles d'entraînement)
    batch_size=32,  # Taille du lot (nombre d'échantillons par mise à jour de gradient)
    validation_data=(X_val, y_val),  # Données de validation pour évaluer le modèle à chaque époque
    callbacks=[early_stop]  # Callback pour arrêter l'entraînement si la perte de validation ne s'améliore plus
)

Epoch 1/10
3990/3990 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/step - accuracy: 0.8031 - loss: 0.1339 - val_accuracy: 0.9941 - val_loss: 0.0742
Epoch 2/10
3990/3990 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - accuracy: 0.9937 - loss: 0.0679 - val_accuracy: 0.9941 - val_loss: 0.0611
Epoch 3/10
3990/3990 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.9942 - loss: 0.0581 - val_accuracy: 0.9941 - val_loss: 0.0580
Epoch 4/10
3990/3990 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.9930 - loss: 0.0524 - val_accuracy: 0.9941 - val_loss: 0.0601
Epoch 5/10
3990/3990 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.9852 - loss: 0.0499 - val_accuracy: 0.9941 - val_loss: 0.0610


## 6.14 Évaluation sur le jeu de validation

In [ ]:
# Prédiction des labels sur le jeu de validation avec le modèle MLP
# Les prédictions sont des probabilités, donc on les convertit en 0 ou 1 avec un seuil de 0.5
y_pred_val_mlp = (model_mlp.predict(X_val) > 0.5).astype("int32")

# Affichage du rapport de classification pour évaluer les performances du modèle MLP
# Le rapport inclut des métriques telles que la précision, le rappel et le score F1 pour chaque classe
print("Rapport de classification - Modèle MLP:")
print(classification_report(y_val, y_pred_val_mlp, target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']))

998/998 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
Rapport de classification - Modèle MLP:
               precision    recall  f1-score   support

        toxic       0.89      0.61      0.73      3056
 severe_toxic       0.71      0.04      0.07       321
      obscene       0.85      0.68      0.75      1715
       threat       0.00      0.00      0.00        74
       insult       0.76      0.58      0.66      1614
identity_hate       0.00      0.00      0.00       294

    micro avg       0.84      0.56      0.68      7074
    macro avg       0.53      0.32      0.37      7074
 weighted avg       0.79      0.56      0.65      7074
  samples avg       0.05      0.05      0.05      7074



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

## 6.15 Test du modèle MLP sur le dataset test.csv

In [11]:
# 1. Chargement et prétraitement des données test
test_data = pd.read_csv('/content/drive/MyDrive/data_classification_commentaires_toxiques/test.csv')
test_data['clean_comment'] = test_data['comment_text'].astype(str).apply(clean_text)

# 2. Conversion des textes en séquences et application du padding
test_sequences = tokenizer.texts_to_sequences(test_data['clean_comment'])
X_test_final = pad_sequences(test_sequences, maxlen=max_len)

# 3. Chargement des labels du fichier test_labels.csv et filtrage des lignes valides (labels != -1)
test_labels = pd.read_csv('/content/drive/MyDrive/data_classification_commentaires_toxiques/test_labels.csv')
valid_idx = (test_labels.iloc[:, 1:] != -1).all(axis=1)  # On garde seulement les lignes avec des labels valides

# Sélection des données et labels valides
X_test_valid = X_test_final[valid_idx]
y_test_valid = test_labels.iloc[:, 1:][valid_idx]

# 4. Prédictions et évaluation du modèle MLP
y_pred_valid_mlp = (model_mlp.predict(X_test_valid) > 0.5).astype("int32")
print("Rapport de classification - Modèle MLP sur test.csv:")
print(classification_report(y_test_valid, y_pred_valid_mlp, target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']))


2000/2000 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
Rapport de classification - Modèle MLP sur test.csv:
               precision    recall  f1-score   support

        toxic       0.63      0.64      0.64      6090
 severe_toxic       0.40      0.05      0.09       367
      obscene       0.73      0.64      0.68      3691
       threat       0.00      0.00      0.00       211
       insult       0.65      0.53      0.58      3427
identity_hate       0.00      0.00      0.00       712

    micro avg       0.66      0.56      0.60     14498
    macro avg       0.40      0.31      0.33     14498
 weighted avg       0.62      0.56      0.58     14498
  samples avg       0.06      0.05      0.05     14498



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

# 6.2 **Modèle B – RNN avec LSTM**
## 6.21 Création du modèle de RNN avec LSTM

On a donc vu précédement les résultats d'un modèle de MLP avec des embeddings, maintenant on va mettre en place un modele LSTM avec embedding

les embeddings étant déjà créé precedement, on crée directement notre modele LSTM

In [ ]:
# Création du modèle LSTM (Long Short-Term Memory)
model_lstm = Sequential([
    # Couche d'embedding pour convertir les indices de mots en vecteurs d'embedding
    Embedding(input_dim=max_words, output_dim=128),  
    
    # Première couche LSTM avec 64 unités, retourne les séquences complètes pour la couche suivante
    LSTM(64, return_sequences=True),  
    
    # Deuxième couche LSTM avec 32 unités, ne retourne que la dernière sortie
    LSTM(32),  
    
    # Couche dense avec 64 neurones et activation ReLU
    Dense(64, activation='relu'),  
    
    # Couche de dropout pour éviter le surapprentissage en désactivant aléatoirement 50% des neurones
    Dropout(0.5),  
    
    # Couche de sortie avec 6 neurones (une pour chaque classe) et activation sigmoïde pour la classification multi-label
    Dense(6, activation='sigmoid')  
])

# Compilation du modèle avec l'optimiseur Adam et la fonction de perte binaire cross-entropie
model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Affichage du résumé du modèle créé
print("Résumé du modèle LSTM :")
model_lstm.summary()

Résumé du modèle LSTM :


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## 6.22 Entraînement du modèle LSTM

In [ ]:
# Entraînement du modèle LSTM avec les données d'entraînement
history_lstm = model_lstm.fit(
    X_train,  # Les séquences de tokens pour l'entraînement
    y_train,  # Les labels multi-labels pour l'entraînement
    epochs=10,  # Nombre d'époques (cycles d'entraînement)
    batch_size=32,  # Taille du lot (nombre d'échantillons par mise à jour de gradient)
    validation_data=(X_val, y_val),  # Données de validation pour évaluer le modèle à chaque époque
    callbacks=[early_stop]  # Callback pour arrêter l'entraînement si la perte de validation ne s'améliore plus
)

Epoch 1/10
3990/3990 ━━━━━━━━━━━━━━━━━━━━ 61s 14ms/step - accuracy: 0.7990 - loss: 0.1189 - val_accuracy: 0.9941 - val_loss: 0.0517
Epoch 2/10
3990/3990 ━━━━━━━━━━━━━━━━━━━━ 79s 14ms/step - accuracy: 0.9928 - loss: 0.0485 - val_accuracy: 0.9941 - val_loss: 0.0500
Epoch 3/10
3990/3990 ━━━━━━━━━━━━━━━━━━━━ 81s 14ms/step - accuracy: 0.9830 - loss: 0.0433 - val_accuracy: 0.9941 - val_loss: 0.0519
Epoch 4/10
3990/3990 ━━━━━━━━━━━━━━━━━━━━ 55s 14ms/step - accuracy: 0.9847 - loss: 0.0385 - val_accuracy: 0.9941 - val_loss: 0.0545


## 6.23 Évaluation sur le jeu de validation

In [ ]:
# Prédiction des labels sur le jeu de validation avec le modèle LSTM
# Les prédictions sont des probabilités, donc on les convertit en 0 ou 1 avec un seuil de 0.5
y_pred_val_lstm = (model_lstm.predict(X_val) > 0.5).astype("int32")

# Affichage du rapport de classification pour évaluer les performances du modèle LSTM
# Le rapport inclut des métriques telles que la précision, le rappel et le score F1 pour chaque classe
print("Rapport de classification - Modèle LSTM:")
print(classification_report(y_val, y_pred_val_lstm, target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']))

998/998 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step
Rapport de classification - Modèle LSTM:
               precision    recall  f1-score   support

        toxic       0.84      0.76      0.80      3056
 severe_toxic       0.52      0.04      0.08       321
      obscene       0.85      0.75      0.80      1715
       threat       0.00      0.00      0.00        74
       insult       0.75      0.61      0.67      1614
identity_hate       0.00      0.00      0.00       294

    micro avg       0.82      0.65      0.73      7074
    macro avg       0.49      0.36      0.39      7074
 weighted avg       0.76      0.65      0.69      7074
  samples avg       0.07      0.06      0.06      7074



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

## 6.24 Test du modèle RNN avec LSTM sur le dataset test.csv

In [15]:
# 1. Chargement et prétraitement des données test
test_data = pd.read_csv('/content/drive/MyDrive/data_classification_commentaires_toxiques/test.csv')
test_data['clean_comment'] = test_data['comment_text'].astype(str).apply(clean_text)

# 2. Conversion des textes en séquences et application du padding
test_sequences = tokenizer.texts_to_sequences(test_data['clean_comment'])
X_test_final = pad_sequences(test_sequences, maxlen=max_len)

# 3. Chargement des labels du fichier test_labels.csv et filtrage des lignes valides (labels != -1)
test_labels = pd.read_csv('/content/drive/MyDrive/data_classification_commentaires_toxiques/test_labels.csv')
valid_idx = (test_labels.iloc[:, 1:] != -1).all(axis=1)  # On garde seulement les lignes avec des labels valides

# Sélection des données et labels valides
X_test_valid = X_test_final[valid_idx]
y_test_valid = test_labels.iloc[:, 1:][valid_idx]

# 5. Prédictions et évaluation du modèle LSTM
y_pred_valid_lstm = (model_lstm.predict(X_test_valid) > 0.5).astype("int32")
print("Rapport de classification - Modèle LSTM sur test.csv:")
print(classification_report(y_test_valid, y_pred_valid_lstm, target_names=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']))

2000/2000 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step
Rapport de classification - Modèle LSTM sur test.csv:
               precision    recall  f1-score   support

        toxic       0.54      0.85      0.66      6090
 severe_toxic       0.41      0.09      0.15       367
      obscene       0.70      0.74      0.72      3691
       threat       0.00      0.00      0.00       211
       insult       0.64      0.60      0.62      3427
identity_hate       0.00      0.00      0.00       712

    micro avg       0.59      0.69      0.64     14498
    macro avg       0.38      0.38      0.36     14498
 weighted avg       0.57      0.69      0.61     14498
  samples avg       0.07      0.07      0.07     14498



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_